In [1]:
!git clone https://github.com/eyalmazuz/DrugWithdrawn.git

Cloning into 'DrugWithdrawn'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 148 (delta 2), reused 0 (delta 0), pack-reused 141
Receiving objects: 100% (148/148), 162.51 MiB | 18.59 MiB/s, done.
Resolving deltas: 100% (63/63), done.
Updating files: 100% (117/117), done.


In [2]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


In [4]:
!pip install accelerate>=0.20.1

In [5]:
!pip install transformers

In [6]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00


In [8]:
# ..............................improved code.......................................

In [9]:
from sklearn.metrics import average_precision_score, roc_auc_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, IntervalStrategy
import torch
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import wandb
from datasets import load_dataset
import evaluate
import pandas as pd
import numpy as np

In [10]:
# Metric loading
metrics = ["roc_auc", "accuracy", "f1", "precision", "recall"]
loaded_metrics = {metric_name: evaluate.load(metric_name) for metric_name in metrics}

In [16]:
# Dataset path generation
def get_dataset_path(split_type, dataset_name, split_name):
    return f'/content/DrugWithdrawn/split/{split_type}/{dataset_name}/{split_name}.csv'

In [19]:
split_type = 'db_no_agree_no_dups'
dataset_name = 'DrugBank'

# Load dataset
dataset = load_dataset('csv', data_files={
    'train': get_dataset_path(split_type, dataset_name, 'train2'),
    'validation': get_dataset_path(split_type, dataset_name, 'val'),
    'test': get_dataset_path(split_type, dataset_name, 'test'),
})

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'smiles', 'length', 'inchikey', 'name', 'groups', 'withdrawn_class', 'source'],
        num_rows: 3198
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'index', 'smiles', 'length', 'inchikey', 'name', 'groups', 'withdrawn_class', 'source'],
        num_rows: 800
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'smiles', 'length', 'inchikey', 'name', 'groups', 'withdrawn_class', 'source'],
        num_rows: 2431
    })
})

In [21]:
# Dataset preparation
dataset = dataset.rename_column('withdrawn_class', 'labels')\
    .remove_columns(['Unnamed: 0', 'index', 'length', 'inchikey', 'groups', 'source'])\
    .with_format('torch')

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['smiles', 'name', 'labels'],
        num_rows: 3198
    })
    validation: Dataset({
        features: ['smiles', 'name', 'labels'],
        num_rows: 800
    })
    test: Dataset({
        features: ['smiles', 'name', 'labels'],
        num_rows: 2431
    })
})

In [30]:
dataset['train']

Dataset({
    features: ['smiles', 'name', 'labels'],
    num_rows: 3198
})

In [31]:
pd.DataFrame(dataset['train'])

,smiles,name,labels
0,CC(C)COC[C@]([H])(CN(Cc1ccccc1)c2ccccc2)N3CCCC3,"bepridil, (s)-",tensor(0)
1,CC12CCCCCC(C1N)CC3=C2C=C(C=C3)O,dezocine,tensor(1)
2,C[C@](Cc1ccc(c(c1)O)O)(C(=O)O)N,methyldopa anhydrous,tensor(0)
3,C1C2CC(C1C=C2)C3NC4=CC(=C(C=C4S(=O)(=O)N3)S(=O...,cyclothiazide,tensor(1)
4,C1CNCC1N2CCC(=CC3=C(N4C(C(C4=O)NC(=O)C(=C5NSC(...,ceftobiprole,tensor(1)
...,...,...,...
3193,C(\[H])(=C(/C#N)\C(=O)O)/c1cc(c(c(c1)O)O)N(=O)=O,entacapone acid,tensor(0)
3194,CCC(=O)O[C@@]1([H])CC[C@@]2(C)C(=CC[C@]3([H])[...,methandriol dipropionate,tensor(1)
3195,Cc1c(cc(C#N)c(n1)O)-c2ccc3nccn3c2.Cl,olprinone hydrochloride,tensor(0)
3196,COc1cc(c(C(=O)CCCN2CCCC2)c(c1)OC)OC.Cl,buflomedil hydrochloride,tensor(0)


In [32]:
pretrained_path = "seyonec/PubChem10M_SMILES_BPE_450k"

tokenizer = AutoTokenizer.from_pretrained(pretrained_path)

model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_path, num_labels=2, id2label={0: 'Not Withdrawn', 1: 'Withdrawn'}, label2id={'Not Withdrawn': 0, 'Withdrawn': 1}
)


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/101k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["smiles"], padding="max_length", truncation=True, max_length=300)

dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3198 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/2431 [00:00<?, ? examples/s]

In [34]:
# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics_dict = {
        **loaded_metrics["f1"].compute(predictions=predictions, references=labels),
        **{'PR-AUC': average_precision_score(y_score=logits[:, 1], y_true=labels)},
        **loaded_metrics["accuracy"].compute(predictions=predictions, references=labels),
        **loaded_metrics["roc_auc"].compute(prediction_scores=logits[:, 1], references=labels),
        **loaded_metrics["precision"].compute(predictions=predictions, references=labels),
        **loaded_metrics["recall"].compute(predictions=predictions, references=labels),
    }
    return metrics_dict

In [35]:
# Training arguments
training_args = TrainingArguments(
    output_dir=f"./results/{split_type}/{dataset_name}/{pretrained_path}",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=6,
    weight_decay=0.01,
    evaluation_strategy=IntervalStrategy.STEPS,
    save_strategy=IntervalStrategy.STEPS,
    report_to='wandb',
    run_name=f'{pretrained_path} {split_type} {dataset_name}',
    logging_steps=50,
    save_steps=50,
)


In [36]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset={'Validation': dataset["validation"], 'Test': dataset["test"]},
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Training
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Validation F1,Validation Pr-auc,Validation Accuracy,Validation Roc Auc,Validation Precision,Validation Recall,Test Loss,Test F1,Test Pr-auc,Test Accuracy,Test Roc Auc,Test Precision,Test Recall
50,0.625500,No log,0.486874,0.679672,0.731250,0.703404,0.886957,0.335526,1.173284,0.212667,0.194608,0.442616,0.659867,0.121838,0.835616
100,0.566900,No log,0.535398,0.699732,0.737500,0.719539,0.817568,0.398026,1.591306,0.205538,0.236365,0.386261,0.680859,0.116335,0.881279
150,0.562200,No log,0.529412,0.726877,0.740000,0.751870,0.847826,0.384868,1.818625,0.210820,0.189906,0.417935,0.676352,0.120076,0.863014
200,0.542100,No log,0.581395,0.691426,0.730000,0.713172,0.707547,0.493421,2.111366,0.190058,0.189147,0.316331,0.670382,0.106383,0.890411
250,0.479600,No log,0.623377,0.750365,0.746250,0.778644,0.714894,0.552632,2.324283,0.206035,0.171725,0.372275,0.680332,0.116265,0.904110
300,0.486400,No log,0.615094,0.751008,0.745000,0.783047,0.721239,0.536184,2.191643,0.210642,0.183657,0.408063,0.688170,0.119701,0.876712
350,0.477600,No log,0.627151,0.757514,0.756250,0.783007,0.748858,0.539474,2.472154,0.199694,0.181976,0.353764,0.679271,0.112385,0.894977
400,0.447100,No log,0.627291,0.780044,0.771250,0.805158,0.823529,0.506579,2.352269,0.211507,0.203523,0.408063,0.700391,0.120174,0.881279
450,0.371400,No log,0.645276,0.758685,0.751250,0.789295,0.704280,0.595395,3.100171,0.186581,0.192245,0.271905,0.679649,0.103730,0.926941
500,0.379400,No log,0.661871,0.778630,0.765000,0.806584,0.730159,0.605263,3.047693,0.191623,0.168110,0.309338,0.675972,0.107104,0.908676


Step,Training Loss,Validation Loss,Validation F1,Validation Pr-auc,Validation Accuracy,Validation Roc Auc,Validation Precision,Validation Recall,Test Loss,Test F1,Test Pr-auc,Test Accuracy,Test Roc Auc,Test Precision,Test Recall
50,0.625500,No log,0.486874,0.679672,0.731250,0.703404,0.886957,0.335526,1.173284,0.212667,0.194608,0.442616,0.659867,0.121838,0.835616
100,0.566900,No log,0.535398,0.699732,0.737500,0.719539,0.817568,0.398026,1.591306,0.205538,0.236365,0.386261,0.680859,0.116335,0.881279
150,0.562200,No log,0.529412,0.726877,0.740000,0.751870,0.847826,0.384868,1.818625,0.210820,0.189906,0.417935,0.676352,0.120076,0.863014
200,0.542100,No log,0.581395,0.691426,0.730000,0.713172,0.707547,0.493421,2.111366,0.190058,0.189147,0.316331,0.670382,0.106383,0.890411
250,0.479600,No log,0.623377,0.750365,0.746250,0.778644,0.714894,0.552632,2.324283,0.206035,0.171725,0.372275,0.680332,0.116265,0.904110
300,0.486400,No log,0.615094,0.751008,0.745000,0.783047,0.721239,0.536184,2.191643,0.210642,0.183657,0.408063,0.688170,0.119701,0.876712
350,0.477600,No log,0.627151,0.757514,0.756250,0.783007,0.748858,0.539474,2.472154,0.199694,0.181976,0.353764,0.679271,0.112385,0.894977
400,0.447100,No log,0.627291,0.780044,0.771250,0.805158,0.823529,0.506579,2.352269,0.211507,0.203523,0.408063,0.700391,0.120174,0.881279
450,0.371400,No log,0.645276,0.758685,0.751250,0.789295,0.704280,0.595395,3.100171,0.186581,0.192245,0.271905,0.679649,0.103730,0.926941
500,0.379400,No log,0.661871,0.778630,0.765000,0.806584,0.730159,0.605263,3.047693,0.191623,0.168110,0.309338,0.675972,0.107104,0.908676


TrainOutput(global_step=1200, training_loss=0.34889670729637146, metrics={'train_runtime': 1419.7204, 'train_samples_per_second': 13.515, 'train_steps_per_second': 0.845, 'total_flos': 1489326823483200.0, 'train_loss': 0.34889670729637146, 'epoch': 6.0})